In [9]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

def encode_categorical(df_train, df_test):
    
    test_start_ind = df_train.index[-1] 
    
    concated_df = pd.concat([df_train, df_test], sort=False) 
        
    encoded_df = pd.get_dummies(concated_df, drop_first=True)
    
    
    df_train_encoded, df_test_encoded = encoded_df.iloc[:test_start_ind, :], encoded_df.iloc[test_start_ind:, :] 
    
    return df_train_encoded, df_test_encoded


def create_a_file(prediction, test_data, name):
    
    result = pd.DataFrame({'PassengerId': test_data.reset_index()['PassengerId'], "Survived": prediction})  
    result.to_csv(name, index=False)

    
path_train = 'C:/Data/train.csv'
path_test = 'C:/Data/test.csv'

train = pd.read_csv(path_train, index_col='PassengerId') 
test = pd.read_csv(path_test, index_col='PassengerId') 

y_train = train['Survived']
train_sep = train.drop(columns=['Survived'])

X_train, X_test = encode_categorical(train_sep, test)

clf = DecisionTreeClassifier()

parameters = {
              'criterion': ['entropy', 'gini'],
              'max_depth': range(1, 10),
              'min_samples_split': range(2, 10),
              'min_samples_leaf': range(1, 10)
             }

rand_grid_cv = RandomizedSearchCV(clf, parameters, cv=5)

rand_grid_cv.fit(X_train, y_train);
model = rand_grid_cv.best_estimator_

rand_grid_pred = model.predict(X_test)



create_a_file(rand_grid_pred, test, 'titanic_dt_gridcv_1.csv')